In [0]:
print("Hello")

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ItemsDataSQL").getOrCreate()


data = [
    ("FDA15", 9.3, "Low Fat", 0.01604730),
    ("DRC01", 5.92, "Regular", 0.01927821),
    ("FDN15", 17.5, "Low Fat", 0.01676007),
    ("FDX07", 19.2, "Regular", 0.0),
    ("NCD19", 8.93, "Low Fat", 0.0),
    ("FDP36", 10.395, "Regular", 0.0),
    ("FDO10", 13.65, "Regular", 0.01274108),
    ("FDP10", None, "Low Fat", 0.12746985),
    ("FDH17", 16.2, "Regular", 0.01668711),
    ("FDU28", 19.2, "Regular", 0.09444959),
    ("FDY07", 11.8, "Low Fat", 0.0),
    ("FDA03", 18.5, "Regular", 0.04546377),
    ("FDX32", 15.1, "Regular", 0.10001315),
    ("FDS46", 17.6, "Regular", 0.04725732)
]

columns = ["Item_identifier", "Item_Weight", "Item_Fat_Content", "Item_Visibility"]

df_items = spark.createDataFrame(data, columns)

df_items.createOrReplaceTempView("items")

# 1. Show all items
spark.sql("SELECT * FROM items").show()

# 2. Items with weight missing (null)
spark.sql("SELECT * FROM items WHERE Item_Weight IS NULL").show()

# 3. Average item weight by fat content
spark.sql("""
    SELECT Item_Fat_Content, AVG(Item_Weight) AS avg_weight
    FROM items
    GROUP BY Item_Fat_Content
""").show()

# 4. Items sorted by visibility descending
spark.sql("SELECT Item_identifier, Item_Visibility FROM items ORDER BY Item_Visibility DESC").show()

# 5. Count of items by fat content
spark.sql("""
    SELECT Item_Fat_Content, COUNT(*) AS count_items
    FROM items
    GROUP BY Item_Fat_Content
""").show()
